In [2]:
import requests
import re
import pickle
import time

In [18]:
api_link = "https://api.github.com/repos"
# issues_link = "/issues"
issues_link = "/issues?state=all"
pull_link = "/pulls"
token_idx = 0
Stop_Flag = False

In [3]:
token_list = pickle.load(open("token.pkl","rb"))

In [4]:
def request_github_api(url):
    global token_idx
    global Stop_Flag
    r = requests.get(url,headers={'Authorization': 'token ' + token_list[token_idx] })
    if int(r.headers['X-RateLimit-Remaining']) > 100 and "200" in r.headers['Status']:
        return r
    else:
#         if token_idx != len(token_list) - 1 and "200" in r.headers['Status']:
        if "200" in r.headers['Status']:
            token_idx = (token_idx + 1) % len(token_list)
            return r
        else:
            print(r.headers)
            Stop_Flag = False
            return r

In [5]:
def next_page_link(req):
    if "Link" in dict(req.headers).keys():
        links = re.sub("<|>|;|,"," ",req.headers["link"]).split()
        try:
            return True,links[links.index("rel=\"next\"") - 1]
        except:
            return False,False        
    else:
        print("No Links")
        return False,False

In [27]:
def extract_github_issue(req_issue_list,issue_dict):
    for issue_sn in range(len(req_issue_list)):
        issue = ''
        issue_json = req_issue_list[issue_sn]
        if issue_json.get("title") != None:
            issue += issue_json['title'] + ".\n "
        if issue_json.get("body") != None:
            issue += issue_json['body'] + ".\n "
        # extract all the link from issue comment
        print(issue_json['comments_url'])
        req_issue_comment = request_github_api(issue_json['comments_url'])
        issue_comment_json = req_issue_comment.json()
        for i in range(len(issue_comment_json)):
            if Stop_Flag == True:
                pickle.dump(issue_dict,open("test.pk","wb"))
                return False
            if issue_comment_json[i].get("body") != None:
                issue += issue_comment_json[i]['body'] + ".\n "
        issue_dict[issue_json['number']] = issue
    return True

In [28]:
def extract_all_issue(url):
    req_issue_list = request_github_api(url)
    if Stop_Flag == True:
        return {}
    issue_dict = {}    
    extract_github_issue(req_issue_list.json(),issue_dict)
    is_next, page_link = next_page_link(req_issue_list)
    while is_next and not Stop_Flag:
        print("is_next")
        print("extract_all_issue " + page_link)
        req_issue_next_page = request_github_api(page_link)
        extract_github_issue(req_issue_next_page.json(),issue_dict)
        is_next, page_link = next_page_link(req_issue_next_page)
        
    return issue_dict

In [1]:
# temp = extract_all_issue(api_link + repo_path + issues_link)
temp = extract_all_issue("https://api.github.com/repos/apigee/istio-mixer-adapter/issues")

In [3]:
repo_list = pickle.load(open("repos_list.pkl","rb"))

In [4]:
repo_list

['https://github.com/vlajos/kubernetes',
 'https://github.com/mjg59/rkt',
 'https://github.com/vsimon/kubernetes',
 'https://github.com/michelle192837/test-infra',
 'https://github.com/odacremolbap/bootkube',
 'https://github.com/dippynark/cert-manager',
 'https://github.com/ramr/origin',
 'https://github.com/lixueclaire/origin',
 'https://github.com/li-ang/client-go',
 'https://github.com/jmccarty3/kubeCloner',
 'https://github.com/k82cn/kubernetes',
 'https://github.com/kubernetes/kompose',
 'https://github.com/gpndata/kubernetes',
 'https://github.com/intfrr/rkt',
 'https://github.com/wikiwi/kube-volume-freezer',
 'https://github.com/pietern/kubernetes',
 'https://github.com/gambol99/node-register',
 'https://github.com/emc-advanced-dev/layerx',
 'https://github.com/wojtek-t/kubernetes',
 'https://github.com/CenturyLinkCloud/adm-forked-kubernetes',
 'https://github.com/sacharya/kubernetes',
 'https://github.com/jhadvig/origin',
 'https://github.com/zefciu/kubernetes',
 'https://gith

In [33]:
for repo in repo_list[:50]:
    repo_path = repo.replace("https://github.com","")
    print(api_link + repo_path + issues_link)
    issue = extract_all_issue(api_link + repo_path + issues_link)
    if len(issue) == 0:
        continue
    else:
        pickle.dump(issue,open("./repo/" + repo_path.replace("/","") ,"wb"))

https://api.github.com/repos/vlajos/kubernetes/issues
No Links
https://api.github.com/repos/mjg59/rkt/issues
No Links
https://api.github.com/repos/vsimon/kubernetes/issues
No Links
https://api.github.com/repos/michelle192837/test-infra/issues
No Links
https://api.github.com/repos/odacremolbap/bootkube/issues
No Links
https://api.github.com/repos/dippynark/cert-manager/issues
No Links
https://api.github.com/repos/ramr/origin/issues
No Links
https://api.github.com/repos/lixueclaire/origin/issues
No Links
https://api.github.com/repos/li-ang/client-go/issues
No Links
https://api.github.com/repos/jmccarty3/kubeCloner/issues
No Links
https://api.github.com/repos/k82cn/kubernetes/issues
No Links
https://api.github.com/repos/kubernetes/kompose/issues
https://api.github.com/repos/kubernetes/kompose/issues/1074/comments
https://api.github.com/repos/kubernetes/kompose/issues/1072/comments
https://api.github.com/repos/kubernetes/kompose/issues/1071/comments
https://api.github.com/repos/kubernetes/

KeyboardInterrupt: 

In [47]:
len(repo_list)

2275

In [31]:
api_link + repo_path + issues_link

'https://api.github.com/repos/noironetworks/aci-containers/issues'

In [45]:
repo.index("")

In [35]:
token_idx

0

'mjg59rkt'

In [6]:
repo_list.index("https://github.com/kubernetes/kubernetes")

308

In [ ]:
temp

In [13]:
len(temp.keys())

402

In [36]:
r1 = request_github_api("https://api.github.com/repositories/113473682/issues?state=all&page=2")

In [37]:
for i in r1.headers:
    print(i + "=> " + r1.headers[i])

Date=> Thu, 01 Nov 2018 18:38:46 GMT
Content-Type=> application/json; charset=utf-8
Transfer-Encoding=> chunked
Server=> GitHub.com
Status=> 200 OK
X-RateLimit-Limit=> 5000
X-RateLimit-Remaining=> 4633
X-RateLimit-Reset=> 1541099035
Cache-Control=> private, max-age=60, s-maxage=60
Vary=> Accept, Authorization, Cookie, X-GitHub-OTP, Accept-Encoding
ETag=> W/"a165e09f6aad8a24ed86ffbbe1ad0e1d"
X-OAuth-Scopes=> admin:org_hook, gist, notifications, read:gpg_key, read:org, read:public_key, read:repo_hook, repo, user
X-Accepted-OAuth-Scopes=> repo
X-GitHub-Media-Type=> github.v3; format=json
Link=> <https://api.github.com/repositories/113473682/issues?state=all&page=1>; rel="prev", <https://api.github.com/repositories/113473682/issues?state=all&page=3>; rel="next", <https://api.github.com/repositories/113473682/issues?state=all&page=14>; rel="last", <https://api.github.com/repositories/113473682/issues?state=all&page=1>; rel="first"
Access-Control-Expose-Headers=> ETag, Link, Location, Retry-

In [25]:
links = re.sub("<|>|;|,"," ",r1.headers['Link']).split()

In [26]:
links

['https://api.github.com/repositories/113473682/issues?state=all&page=1',
 'rel="prev"',
 'https://api.github.com/repositories/113473682/issues?state=all&page=3',
 'rel="next"',
 'https://api.github.com/repositories/113473682/issues?state=all&page=14',
 'rel="last"',
 'https://api.github.com/repositories/113473682/issues?state=all&page=1',
 'rel="first"']

In [30]:
links.index("rel=\"next\"") - 1

2

In [31]:
r2 = request_github_api("https://api.github.com/repositories/113473682/issues?state=all&page=14")
for i in r2.headers:
    print(i + "=> " + r2.headers[i])

Server=> GitHub.com
Date=> Thu, 01 Nov 2018 07:28:39 GMT
Content-Type=> application/json; charset=utf-8
Transfer-Encoding=> chunked
Status=> 200 OK
X-RateLimit-Limit=> 5000
X-RateLimit-Remaining=> 4727
X-RateLimit-Reset=> 1541057860
Cache-Control=> private, max-age=60, s-maxage=60
Vary=> Accept, Authorization, Cookie, X-GitHub-OTP
ETag=> W/"927375ec55f87b6732a8fd52723fab12"
X-OAuth-Scopes=> admin:org_hook, gist, notifications, read:gpg_key, read:org, read:public_key, read:repo_hook, repo, user
X-Accepted-OAuth-Scopes=> repo
X-GitHub-Media-Type=> github.v3; format=json
Link=> <https://api.github.com/repositories/113473682/issues?state=all&page=13>; rel="prev", <https://api.github.com/repositories/113473682/issues?state=all&page=1>; rel="first"
Access-Control-Expose-Headers=> ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type
Access-Control-Allow-Origin

In [32]:
links_2 = re.sub("<|>|;|,"," ",r2.headers['Link']).split()

ValueError: 'rel="next"' is not in list

In [2]:
repo_list = pickle.load(open("repos_list.pkl","rb"))


In [4]:
len(repo_list)

2275